In [ ]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import trimesh
from skimage import measure
import meshplot as mp
from torch.utils.data import DataLoader, Dataset
import os
from datetime import datetime

In [ ]:
# tag shape index to each x=xyz, y=sdf
# horizontally concatenate for multiple chapes
# returns shape index, x=xyz (1 by 3), y=sdf
class ChairDataset(Dataset):
    def __init__(self, file_paths):
        for idx, file_path in enumerate(file_paths): #TODO: move to getitem
            training_set = np.load(file_path)
            points = training_set['points']
            points = torch.from_numpy(points.astype(np.float32)) #TODO: dypte precision is lost??
            idx_tensor = torch.ones((points.shape[0], 1), dtype=torch.int) * (idx) # 0 based indexing shapes
            
            sdf = training_set['sdf']
            sdf = torch.from_numpy(sdf.astype(np.float32)) 
            sdf = sdf.view(sdf.shape[0], 1)
            
            n_samples = sdf.shape[0] 

            if idx == 0:
                self.shape_idx = idx_tensor
                self.x = points
                self.y = sdf
                self.n_samples = n_samples
            else:
                self.shape_idx = torch.cat((self.shape_idx, idx_tensor), dim = 0) # concatenate vertically
                self.x = torch.cat((self.x, points), dim = 0) #concatenate vertically
                self.y = torch.cat((self.y, sdf), dim = 0) #concatenate vertically
                self.n_samples += n_samples
            
    def __getitem__(self, index):
        return self.shape_idx[index], self.x[index], self.y[index]
    
    def __len__(self):
        return self.n_samples

In [ ]:
class MLP(nn.Module):
    def __init__(self, n_shapes, shape_code_length, n_inner_nodes):
        super(MLP, self).__init__()
        self.shape_code_length = shape_code_length
        self.shape_codes = nn.Embedding(n_shapes, shape_code_length, max_norm=0.01) # shape code as an embedding # TODO: take this outside 
        self.linear1 = nn.Linear(3 + shape_code_length, n_inner_nodes) # (x, y, z) + shape code 
        self.linear2 = nn.Linear(n_inner_nodes, n_inner_nodes)
        self.linear3 = nn.Linear(n_inner_nodes, 1) # output a SDF value
        self.relu = nn.ReLU()

    def forward(self, shape_idx, x):
        shape_code = self.shape_codes(shape_idx.view(1, -1))
        shape_code = shape_code.view(-1, self.shape_code_length)
        shape_code_with_xyz = torch.cat((x, shape_code), dim=1) # concatenate horizontally
        
        out = self.linear1(shape_code_with_xyz)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.relu(out)
        out = self.linear3(out)
        
        return out

In [ ]:
n_files = 10
file_paths = []
main_dir = '../data/03001627_sdfs/'
for sub_dir in os.scandir(main_dir):
    if sub_dir.is_dir():
        for file in os.listdir(main_dir + sub_dir.name):
            file_paths.append(main_dir + sub_dir.name + '/' + file) if file.endswith("sdf_samples.npz") else None
    if len(file_paths) == n_files:
        break

print(file_paths)

In [ ]:
dataset = ChairDataset(file_paths)
dataloader = DataLoader(dataset=dataset, batch_size=2048, shuffle=True) #TODO: can specify num_workers

n_epochs = 10000
learning_rate = 0.001
# hidden_size = 500 
# hidden_layer_act_func = "ReLU"
# output_layer_act_func = None

model = MLP(len(file_paths), 256, 256)

criterion = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model.train(True) # prep model for training, needed for modules like Dropout

now = datetime.now() 
now = now.strftime("%m%d%Y_%H%M%S")

for epoch in range(n_epochs):
    train_loss = 0.0 # monitor training loss

    for shape_idx, xyz, sdf in dataloader:
        optimizer.zero_grad() 
        sdf_pred = model(shape_idx, xyz) 
        loss = criterion(torch.clamp(sdf_pred, -0.1, 0.1), torch.clamp(sdf, -0.1, 0.1)) 
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()*shape_idx.size(0) # update running training loss
        
    train_loss = train_loss/len(dataloader.dataset) # calculate average loss over one epoch
    if epoch%10 == 0:
        print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))
        torch.save(model.state_dict(), f'./models/multiple_shapes/' + now)

In [ ]:
filename = f'./models/multiple_shapes'
loaded_model = MLP(len(file_paths), 256, 256)
loaded_model.load_state_dict(torch.load(filename))
loaded_model.eval()

x = np.linspace(-1, 1, 150, dtype=np.float32)
y = np.linspace(-1, 1, 150, dtype=np.float32)
z = np.linspace(-1, 1, 150, dtype=np.float32)
P = np.vstack(np.meshgrid(x,y,z)).reshape(3,-1).T  # format: [[x1, y1, z1], [x1, y1, z2], [] ...]
P = torch.from_numpy(P)

shape_idx_tensor = torch.ones((P.shape[0], 1), dtype=torch.int) * (0)

volume = loaded_model(shape_idx_tensor,P).view(len(x), len(y), len(z)) #TODO: double check & compare numpy reshape vs pytorch view
volume = volume.detach().numpy()

verts, faces, normals, values = measure.marching_cubes(volume, 0)
mp.plot(verts, faces)

In [ ]:
# compare with ground truth
mesh = trimesh.load(file_paths[0].split('sdf_samples.npz')[0] + 'mesh.obj')
mp.plot(mesh.vertices, mesh.faces)